In [6]:
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
import warnings
import joblib

from utils import make_dataset, display_results

warnings.filterwarnings('ignore')

PRIVILEGED_INFO = {'unprivileged_groups':[{'race': 2},
                                          {'race': 1},
                                          {'race': 4},
                                          {'race': 5},
                                          {'race': 6}],
                   'privileged_groups':[{'race': 3}]
                  }

BIAS_INFO = {'favorable_label':0,
             'unfavorable_label':1,
             'protected_columns':['race'],
            }

data = pd.read_csv('../data/processed/compas-scores-two-years-processed.csv')

DROP_COLS = ['two_year_recid','compas_score','decile_score','compas_class']
FEATURE_COLS = data.drop(DROP_COLS, axis=1).columns.tolist()

train, test = train_test_split(data, test_size=0.2, random_state=1234)

,age,juv_fel_count,juv_misd_count,juv_other_count,priors_count,jail_time,sex,race,c_charge_degree,decile_score,two_year_recid,compas_score
0,69,0,0,0,0,0.0,1,1,1,1,0,0.1
1,34,0,0,0,0,10.0,1,2,1,3,1,0.3
2,24,0,0,1,4,1.0,1,2,1,4,1,0.4
3,23,0,1,0,1,0.0,1,2,1,8,0,0.8
4,43,0,0,0,2,0.0,1,1,1,1,0,0.1


In [8]:
from sklearn.preprocessing import MinMaxScaler

def collapse_group(x):
    if x!=3:
        return 0
    else:
        return 1

train['race'] = train['race'].apply(collapse_group)
test['race'] = test['race'].apply(collapse_group)

PRIVILEGED_INFO = {'unprivileged_groups':[{'race': 0}],
                   'privileged_groups':[{'race': 1}]
                  }

scaler = MinMaxScaler()
train[FEATURE_COLS] = scaler.fit_transform(train[FEATURE_COLS])
test[FEATURE_COLS] = scaler.transform(test[FEATURE_COLS])

In [9]:
ground_truth_train = make_dataset(train[FEATURE_COLS], train['two_year_recid'], **BIAS_INFO, **PRIVILEGED_INFO)
ground_truth_test = make_dataset(test[FEATURE_COLS], test['two_year_recid'], **BIAS_INFO, **PRIVILEGED_INFO)

# Model fairness logger 

With one function call:

```python
log_fairness_classification_metrics(test['two_year_recid'], test['class_predicted'], test[['race']],
                                    favorable_label=0, unfavorable_label=1,
                                    privileged_groups = {'race':[1]}, unprivileged_groups={'race':[0]})
```

You can log all those metrics:

<img src="../images/fairness_metrics.gif" alt="Drawing" style="width: 600px;"/>

In [20]:
import neptune
from neptunecontrib.monitoring.fin-processingairness import log_fairness_classification_metrics

from aif360.algorithms.inprocessing import AdversarialDebiasing
import tensorflow as tf

neptune.init('shared/model-fairness-workshop')

model_params = {'classifier_num_hidden_units':200, 
                'num_epochs':50,
                'batch_size':128,
                'adversary_loss_weight':0.1,
                'debias':True
               }

neptune.create_experiment(name='compas', tags=['race'], 
                          params=model_params,
                          upload_source_files=['*.py','*.ipynb'])

sess = tf.Session()

clf = AdversarialDebiasing(scope_name='debiased_classifier', sess=sess, seed=1234, 
                           **PRIVILEGED_INFO,
                           **model_params
                           )

clf.fit(ground_truth_train)
prediction_test = clf.predict(ground_truth_test)

roc_auc = roc_auc_score(test[TARGET_COLS], prediction_test.labels)

neptune.log_metric('roc_auc',roc_auc)
log_fairness_classification_metrics(test['two_year_recid'], test['class_predicted'], test[['race']],
                                    favorable_label=0, unfavorable_label=1,
                                    privileged_groups = {'race':[1]}, unprivileged_groups={'race':[0]})

neptune.stop()

https://ui.neptune.ml/shared/model-fairness-workshop/e/MOD-8
epoch 0; iter: 0; batch classifier loss: 0.691432; batch adversarial loss: 0.867698
epoch 1; iter: 0; batch classifier loss: 0.668847; batch adversarial loss: 0.807987
epoch 2; iter: 0; batch classifier loss: 0.648525; batch adversarial loss: 0.839774
epoch 3; iter: 0; batch classifier loss: 0.656971; batch adversarial loss: 0.885115
epoch 4; iter: 0; batch classifier loss: 0.645133; batch adversarial loss: 0.803704
epoch 5; iter: 0; batch classifier loss: 0.634000; batch adversarial loss: 0.806883
epoch 6; iter: 0; batch classifier loss: 0.684488; batch adversarial loss: 0.760037
epoch 7; iter: 0; batch classifier loss: 0.638033; batch adversarial loss: 0.740264
epoch 8; iter: 0; batch classifier loss: 0.640675; batch adversarial loss: 0.717860
epoch 9; iter: 0; batch classifier loss: 0.649522; batch adversarial loss: 0.718971
epoch 10; iter: 0; batch classifier loss: 0.558831; batch adversarial loss: 0.713247
epoch 11; iter

In [19]:
sess.close()
tf.reset_default_graph()